In [4]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

import tweepy
import newspaper

import gensim
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from stop_words import get_stop_words

from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import pandas as pd
import numpy as np

import re
from time import sleep

import pickle
config

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pprint

def SaveDict(filename, mode, root):
    if filename[-4:]!=".txt":
        filename+=".txt"
    with open(filename, mode) as f:
        f.write(pprint.pformat(root))

def LoadDict(filename):
    if filename[-4:]!=".txt":
        filename+=".txt"
    with open(filename, "r") as f:
        return eval(f.read())

In [3]:
def twitter_profile(screen_name,api,num_tweets=1000):
    
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=screen_name, count=200, include_rts=True).pages(num_tweets/200):
        tweets += tweet
    tweets = [[tweet.text,[url['expanded_url'] for url in tweet.entities['urls']]] for tweet in tweets]

    return tweets

In [ ]:
auth = tweepy.AppAuthHandler(config["consumer_key"], config["consumer_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
tweets = twitter_profile("huangy22",api)

In [ ]:
def twitter_links(tweets,conn):
        
    url1 = pub_twitter['short_url'][pub_twitter['short_url'].notnull() 
                     & ~(pub_twitter['short_url'].isin(['http','bit.ly','ow.ly','buff.ly',
                                                         'trib.al','yhoo.it','buzzfeed.com']))].values.tolist() 
    url2 = pub_twitter['long_url'].values.tolist()
    pattern = '|'.join(map(re.escape,url1+url2))
    
    tweets = pd.DataFrame(tweets)
    tweets.columns = ['text','url']
    tweets['url'] = tweets['url'].apply(lambda x: ', '.join(x))
    
    links = tweets['url'][tweets['url'].str.contains(pattern)].values.tolist()
    links = [link.split(',')[0] for link in links]
    
    return links,tweets